In [1]:
import pandas as pd
import numpy as np
from numpy import array
from numpy import savetxt
from scipy.ndimage.interpolation import shift

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Dropout, LSTM, Conv1D
from tensorflow.keras.layers import Flatten
import matplotlib.pyplot as plt


In [2]:
#Loading data from PC 
col_list = ["Actual Irr"]
data = pd.read_csv("PDNsolar.csv", usecols=col_list)
data = data.rename(columns={"Actual Irr" : "irradiance" }) #Change Column title
print(data.head())

   irradiance
0        0.00
1        0.00
2        0.00
3        3.76
4       21.72


In [3]:
#Assign data to numpy array
datamat = data['irradiance'].to_numpy()
datamat = datamat.reshape(len(datamat),1)

In [4]:
#Normalise data
mu = np.mean(datamat)
sigma = np.std(datamat)
datanorm = (datamat-mu)/sigma

In [5]:
#Train and test set split
trainval = int(np.floor(0.75*len(datanorm)))
train_data = datanorm[0:trainval]
test_data = datanorm[trainval:]

In [6]:
#Split into input ouput sets
def split_seq(seq, n_steps):
    X, y = list(), list()
    for i in range(len(seq)):
    
        #Find end of seq
        end_ix = i+n_steps
        #Check if sequence passed
        if end_ix>len(seq)-1:
            break

        #Gather input and ouputs
        seq_x, seq_y = seq[i:end_ix], seq[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)


In [7]:
#No. of steps
n_steps = 3

#Split into inputs, outputs for training data
Xtrain, Ytrain = split_seq(train_data, n_steps)

n_features = 1
Xtrain = Xtrain.reshape(Xtrain.shape[0],Xtrain.shape[1], n_features)

#Split into input and outputs for test data
Xtest, Ytest = split_seq(test_data, n_steps)

Xtest = Xtest.reshape(Xtest.shape[0], Xtest.shape[1], n_features)

In [8]:
#Define Model
model = tf.keras.Sequential()
model.add(LSTM(10, input_shape=(n_steps, n_features), activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))
model.compile(optimizer=tf.keras.optimizers.Adam(0.001), loss='mse')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 10)                480       
_________________________________________________________________
dense (Dense)                (None, 128)               1408      
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 10,209
Trainable params: 10,209
Non-trainable params: 0
_________________________________________________________________


In [9]:
#Fit model
history = model.fit(Xtrain, Ytrain, epochs=10, batch_size=64, validation_split=0.15, verbose=1, shuffle='false')

Train on 18196 samples, validate on 3212 samples
Epoch 1/10
18196/18196 [==============================] - 6s 319us/sample - loss: 0.2255 - val_loss: 0.1636
Epoch 2/10
18196/18196 [==============================] - 3s 151us/sample - loss: 0.1589 - val_loss: 0.1479
Epoch 3/10
18196/18196 [==============================] - 3s 151us/sample - loss: 0.1555 - val_loss: 0.1504
Epoch 4/10
18196/18196 [==============================] - 3s 150us/sample - loss: 0.1542 - val_loss: 0.1448
Epoch 5/10
18196/18196 [==============================] - 3s 151us/sample - loss: 0.1532 - val_loss: 0.1457
Epoch 6/10
18196/18196 [==============================] - 3s 149us/sample - loss: 0.1528 - val_loss: 0.1450
Epoch 7/10
18196/18196 [==============================] - 3s 150us/sample - loss: 0.1519 - val_loss: 0.1434
Epoch 8/10
18196/18196 [==============================] - 3s 151us/sample - loss: 0.1513 - val_loss: 0.1455
Epoch 9/10
18196/18196 [==============================] - 3s 150us/sample - loss: 0.150

In [ ]:
# plt.plot(a)

In [88]:
#Preparing Entire dataset for the network
X, Y = split_seq(datanorm, n_steps)

In [89]:
#Predict using model
yhat_test = model.predict(Xtest, verbose=0)
yhat = model.predict(X, verbose=0)

In [90]:
#Regain actual data val
ypred_test = yhat_test*sigma + mu
ypred = yhat*sigma + mu
Ytest = Ytest*sigma + mu
Y = Y*sigma + mu

In [91]:
#Stack predicted and actual irradiance
Yout_test = np.hstack((Ytest, ypred_test)) #Test data only
Yout = np.hstack((Y, ypred)) #All data

In [92]:
RMSE = np.sqrt(np.sum((Ytest-ypred_test)**2)/len(Ytest))
RMSEall = np.sqrt(np.sum((Y-ypred)**2)/len(Y))
print(f"The test set RMSE is {RMSE}")
print(f"The entire set RMSE is {RMSEall}")

The test set RMSE is 107.65856708957146
The entire set RMSE is 117.20000967034159


In [ ]:
#Save file to csv
savetxt('LSTM_yout_test128_64.csv', Yout_test, delimiter=',')
savetxt('LSTM_yout128_64.csv', Yout, delimiter=',')